In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
train_data = pd.read_csv('C:/Users/KIIT0001/Desktop/MAHASWETA/ML-INTERNSHIP/fraudTrain.csv')
train_data.head(2)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0


In [3]:
test_data = pd.read_csv('C:/Users/KIIT0001/Desktop/MAHASWETA/ML-INTERNSHIP/fraudTest.csv')
test_data.head(2)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0


In [4]:
# merging train and test data
data = pd.concat([train_data, test_data], axis=0)  

# reset index
data.reset_index(drop=True, inplace=True)
data.head(2)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0


# Data pre-processing

In [5]:
data.info

<bound method DataFrame.info of          Unnamed: 0 trans_date_trans_time            cc_num  \
0                 0   2019-01-01 00:00:18  2703186189652095   
1                 1   2019-01-01 00:00:44      630423337322   
2                 2   2019-01-01 00:00:51    38859492057661   
3                 3   2019-01-01 00:01:16  3534093764340240   
4                 4   2019-01-01 00:03:06   375534208663984   
...             ...                   ...               ...   
1852389      555714   2020-12-31 23:59:07    30560609640617   
1852390      555715   2020-12-31 23:59:09  3556613125071656   
1852391      555716   2020-12-31 23:59:15  6011724471098086   
1852392      555717   2020-12-31 23:59:24     4079773899158   
1852393      555718   2020-12-31 23:59:34  4170689372027579   

                                   merchant        category     amt  \
0                fraud_Rippin, Kub and Mann        misc_net    4.97   
1           fraud_Heller, Gutmann and Zieme     grocery_pos  107.23  

In [6]:
data.shape

(1852394, 23)

In [7]:
data.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06
mean,5.371934e+05,4.173860e+17,7.006357e+01,4.881326e+04,3.853931e+01,-9.022783e+01,8.864367e+04,1.358674e+09,3.853898e+01,-9.022794e+01,5.210015e-03
std,3.669110e+05,1.309115e+18,1.592540e+02,2.688185e+04,5.071470e+00,1.374789e+01,3.014876e+05,1.819508e+07,5.105604e+00,1.375969e+01,7.199217e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902742e+01,-1.666716e+02,0.000000e+00
25%,2.315490e+05,1.800429e+14,9.640000e+00,2.623700e+04,3.466890e+01,-9.679800e+01,7.410000e+02,1.343017e+09,3.474012e+01,-9.689944e+01,0.000000e+00
50%,4.630980e+05,3.521417e+15,4.745000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.443000e+03,1.357089e+09,3.936890e+01,-8.744069e+01,0.000000e+00
75%,8.335758e+05,4.642255e+15,8.310000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.374581e+09,4.195626e+01,-8.024511e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.992100e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.388534e+09,6.751027e+01,-6.695090e+01,1.000000e+00


# Handling missing values

In [8]:
data.isnull().sum()
# no missing value present

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

# Label Encoding

In [24]:
print(data.dtypes)

Unnamed: 0                 int64
trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object


In [26]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
le = LabelEncoder()

# Convert categorical (object) columns to numerical using Label Encoding
categorical_cols = ['trans_date_trans_time', 'merchant', 'category', 'first', 'last', 'gender',
                    'street', 'city', 'state', 'job', 'dob', 'trans_num']

for col in categorical_cols:
    data[col] = le.fit_transform(data[col])

# Convert all int columns to float
int_cols = ['cc_num', 'zip', 'city_pop', 'unix_time', 'is_fraud']
data[int_cols] = data[int_cols].astype(float)

# Convert entire dataset to float
data = data.astype(float)

print(data.dtypes)

Unnamed: 0               float64
trans_date_trans_time    float64
cc_num                   float64
merchant                 float64
category                 float64
amt                      float64
first                    float64
last                     float64
gender                   float64
street                   float64
city                     float64
state                    float64
zip                      float64
lat                      float64
long                     float64
city_pop                 float64
job                      float64
dob                      float64
trans_num                float64
unix_time                float64
merch_lat                float64
merch_long               float64
is_fraud                 float64
dtype: object


In [35]:
data.head(3)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0.0,0.0,2.703186e+15,514.0,8.0,4.97,164.0,18.0,0.0,576.0,...,36.0788,-81.1781,3495.0,372.0,791.0,80326.0,1.325376e+09,36.011293,-82.048315,0.0
1,1.0,1.0,6.304233e+11,241.0,4.0,107.23,312.0,161.0,0.0,439.0,...,48.8878,-118.2105,149.0,431.0,619.0,227462.0,1.325376e+09,49.159047,-118.186462,0.0
2,2.0,2.0,3.885949e+13,390.0,0.0,220.11,116.0,386.0,1.0,610.0,...,42.1808,-112.2620,4154.0,308.0,309.0,1169030.0,1.325376e+09,43.150704,-112.154481,0.0


# Distribution of legit transactions & fraudulent transactions

In [27]:
data['is_fraud'].value_counts()

is_fraud
0.0    1842743
1.0       9651
Name: count, dtype: int64

In [28]:
# 0 -> legitimate transaction
# 1 -> fradulent transaction

In [29]:
# separating the data for analysis
legit = data[data.is_fraud == 0]
fraud = data[data.is_fraud == 1]

In [30]:
print(legit.shape)
print(fraud.shape)

(1842743, 23)
(9651, 23)


# Splitting data & festures

In [31]:
X = data.drop(columns='is_fraud', axis=1)
Y = data['is_fraud']

print(X)
print(Y)

         Unnamed: 0  trans_date_trans_time        cc_num  merchant  category  \
0               0.0                    0.0  2.703186e+15     514.0       8.0   
1               1.0                    1.0  6.304233e+11     241.0       4.0   
2               2.0                    2.0  3.885949e+13     390.0       0.0   
3               3.0                    3.0  3.534094e+15     360.0       2.0   
4               4.0                    4.0  3.755342e+14     297.0       9.0   
...             ...                    ...           ...       ...       ...   
1852389    555714.0              1819546.0  3.056061e+13     507.0       5.0   
1852390    555715.0              1819547.0  3.556613e+15     264.0       7.0   
1852391    555716.0              1819548.0  6.011724e+15     496.0       7.0   
1852392    555717.0              1819549.0  4.079774e+12      75.0      13.0   
1852393    555718.0              1819550.0  4.170689e+15     125.0       0.0   

            amt  first   last  gender  

# Splitting train-test data

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)
print(X.shape, X_train.shape, X_test.shape)

(1852394, 22) (1481915, 22) (370479, 22)


# Training model

In [33]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

# Model evalustion

In [34]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy on Training data : ', training_data_accuracy)

# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy on Training data :  0.9947898496202549
Accuracy score on Test Data :  0.9947905279381557


# Predictive system

In [40]:
input_data = (0,0.0,0.0,2.703186e+15,514.0,8.0,4.97,164.0,18.0,0.0,576.0,0,36.0788,-81.1781,3495.0,372.0,791.0,80326.0,1.325376e+09,36.011293,-82.048315,0.0)

# changing the input data to numpy array
numpy_array_of_input_data = np.asarray(input_data)

# reshape the numpy array as we are predicting for one instance
input_data_reshaped = numpy_array_of_input_data.reshape(1,-1)

# make a prediction
prediction = model.predict(input_data_reshaped)
print(prediction)

if(prediction[0] == 1):
    print("Legitimate")
else:
    print("Fraud")

[0.]
Fraud


C:\Users\KIIT0001\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
